In [9]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
import numpy as np

In [2]:
df = pd.read_csv('../data/housing.csv')

In [3]:
df


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    int64  
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  PRICE    506 non-null    float64
dtypes: float64(11), int64(3)
memory usage: 55.5 KB


***

# Recap of last time

* Started by looking at decision trees - as these are the building blocks of random forests
* We used a general data set that looked at Boston Housing


In [4]:
tree = DecisionTreeRegressor() # Initialise

## So what do trees do?

* Split the data to get the target data so that the averages of the target variable on each side of the split are as close to each other as possible
* Close in this case means that that the averages on each side of the split are as close to each other as possible

In [27]:
left = df[df.LSTAT < 9.67]
right = df[df.LSTAT >= 9.67]

In [28]:
# Error column using average of full data set to look at the values
df['PRICE'] - df['PRICE'].mean()

0       1.467194
1      -0.932806
2      12.167194
3      10.867194
4      13.667194
         ...    
501    -0.132806
502    -1.932806
503     1.367194
504    -0.532806
505   -10.632806
Name: PRICE, Length: 506, dtype: float64

In [29]:
# Root mean squared error (i.e. average of square of errors)
root_error = np.mean((df['PRICE'] - df['PRICE'].mean())**2)

In [30]:
root_error

84.4195561561656

In [31]:
right_error = np.mean((right['PRICE'] - right['PRICE'].mean())**2)
left_error = np.mean((left['PRICE'] - left['PRICE'].mean())**2)

In [32]:
right_error # Right side error is better

24.32308736092969

In [33]:
left_error # Left side error is a lot better

79.95297868897926

So this means that both the left and right side of the data set are more similar to each other in a meaningful way.
* This is shown because the Root Error of each side of the split is smaller than the average for the node above (i.e. full data set)

In [34]:
num_samples = df.shape[0]
right_samples = right.shape[0]
left_samples = left.shape[0]

In [35]:
num_samples

506

In [36]:
right_samples

298

In [37]:
left_samples

208

In this case our split is very lopsided on each side.
* So we also need to make sure we scale the error terms to see if things have actually improved.

In [38]:
scaled_right_error = right_samples / num_samples * right_error

In [39]:
scaled_left_error = left_samples / num_samples * left_error

In [40]:
scaled_right_error

14.32466409793883

In [41]:
scaled_left_error

32.86604657570689

The decision tree then compares these weighted samples to the original root error.

In [42]:
root_error - (right_samples / num_samples * right_error) - (left_samples / num_samples * left_error)

37.22884548251987

So this is the criteria will test to see if it should split on a particular value.
If the root_error will remain unchanged. This implies a bigger number is better. 
* Concept **INFORMATION GAIN** so we have gained information.

Decision trees tend to prefer:
* Things that split the sample in the node equally
* Splits that improve the information gain (minimise errors in the samples)

The tree will keep splitting until there is nothing left to split on.

## Testing a new value - for the next node in the tree


In [ ]:
# For example a split of the initial node 'Tax'
left = left[left.Tax <273]
right = left[left.Tax >=273]

***

## Big benefits of decision trees

* Don't need to make any assumptions about the underlying data
    * Scale doesn't matter
    * Formatting doesn't matter
    * Only looks at whether on left or right side - magnitude doesn't influence this
    * Somewhat immune to outliers

* This creates an advantage over standard statistical methods
    * Typical statistical approaches make bigt assumptions

* We can just take a data set and fit a model on it as it is.


## Shortcomings of decision trees

**Good at 'memorising' data sets**

* Often have trouble with out of sample data
* The deeper you get into a tree, the less reliable the splits become
    * You can get quite different splits if you just change the order
* Tend to overfit data sets, since it just operates recursively
* Typically beyond 6-8 levels, reproduciability becomes very erratic

## Questions

***
*q: How does a tree decide what to sample first?*

It will look at each unique value in each column, and then returns the one with the highest value of each expression. It's a 'double for loop'

***
*q: Will it always be the same?*

Yes - in standard decision tree given that every unique value is considered to maximise information gain. A variety of techniques introduce additional randomness.

***
*q: ?*

*** 
# Talking about sci-kit learn

`import sklearn`


**scikit-learn.org**

* This is the main library we will be using for Unit 3
* Fairly well developed, generally the most popular machine learning library in python
* Covers most of the main machine-learning model families
* Be aware that scikit-learn operates on a CPU vs a GPU


* It has a simple and straightforward API syntax
* Most other machine learning packages in python copy scikit-learn


## Major methods in scikit-learn

* fit() - fits a data set to the target variable
* score()
* predict() - estimates outputs using the tree


* get_params() - gives parameters of model

In [44]:
# initiatlise your algorithm

tree = DecisionTreeRegressor(max_depth=4)

In [47]:
# declare X & y (X - predictors / y - target variable; thing we're precdicting)

X = df.drop('PRICE',axis = 1)
X = df.drop(['PRICE','Prediction'],axis = 1)
y = df['PRICE']

# this is the learning bit - it figures out the values of the nodes
tree.fit(X,y)

DecisionTreeRegressor(max_depth=4)

In [49]:
# this will go ahead and make predictions
df['Prediction'] = tree.predict(X)

# This takes each unique sample and places it on it's corresponding leaf.
# i.e: runs it through the tree

In [51]:
# So now have PRICE and Prediction (what our tree thinks it should be)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,Prediction
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0,27.427273
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6,21.629744
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7,32.748780
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4,32.748780
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2,32.748780


In [59]:
# Example of loop:
max_gain = 0
col_split = None

for col in df.columns:

    for value in df[col].unique():
        left = df[df[col] < value]
        right = df[df[col] >= value]
        
        root_error = np.mean(df['PRICE'] - df['PRICE'].mean()**2)
        left_error = np.mean(left['PRICE'] - left['PRICE'].mean()**2)
        right_error = np.mean(right['PRICE'] - right['PRICE'].mean()**2)
        
        info_gain = root_error - (right.shape[0]/df.shape[0]) * right_error - (left.shape[0] / df.shape[0])*left_error
        
        if info_gain > max_gain:
            max_gain = info_gain
            col_split = col

In [61]:
max_gain

53.41490142053823

In [62]:
col_split

'PRICE'

In [63]:
tree.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': 4,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': 'deprecated',
 'random_state': None,
 'splitter': 'best'}

In [65]:
# score provides the R-square value
# estimate of how well the model fits the data
tree.score(X,y)

0.8857396443908376

***

# Tree Based Ensembles

* These attempt to get around the issues around producing predictions with decision trees.


* We will focus on talking about **Gradient Boosting** ensemble techniques.
    * Tends to work better than the theory behind it would predict
    * Often not covered in masters courses (at best a week or two)
    * *VERY GOOD IN THE REAL WORLD!*
    


## What is boosting?

* A general purpose machine learning techniques
* Combines many weak models into something more powerful
* Boosting...
    * Is **additive**: Fit lots of small models, then combine them together.
    * Is **adaptive**: Every subsequent model pays more attention to the answers it got wrong on the last model.
    
## What is gradient boosting?

**Gradient -** fancy word for errors
* A specific approach to boosting models


## Benefits of Gradient boosting
* Uses trees as the base models so tend to get the benefits of these trees
* Can wrap themselves around non-lineaer features of the data set
* Can with a bit of tuning become fairly robust
* Can fit to data sets of arbitrary size


## Other notes

* Tree's we are fitting are not all that deep
* Can fit as many as you like, over and over again (in theory infinite!)
* This will allow them to adapt and combine to fit an increasingly complex data set
* non-parametric

`feature_importances_` : tells us how much each column contributed to a model score.


## Example of boosting:

In [71]:
# X = df.drop(['PRICE'],axis = 1)
X = df.drop(['PRICE','Prediction'],axis = 1)
y = df['PRICE']


In [72]:
# boosting starts with a NAIVE prediction
# typically for regression the avg value of y
naive_guess = y.mean()

In [78]:
# get the error column
gradient = y - y.mean() # Fancy term for error column

In [79]:
# initialise a decision tree -- it will usually be shallow
tree = DecisionTreeRegressor(max_depth=4)

In [80]:
# fit a tree on X & **the gradient**
# Try to predict the errors, predicting the difference from the average
tree.fit(X,gradient)

DecisionTreeRegressor(max_depth=4)

In [82]:
# This gives our predicted amount error
tree.predict(X)

# This gives a prediction on HOW DIFFERENT/HOW FAR AWAY from the 
# average value of y this sample will be.

array([  4.8944664 ,  -0.90306273,  10.21597416,  10.21597416,
        10.21597416,  -0.90306273,  -0.90306273,  -2.51197299,
        -2.51197299,  -2.51197299,  -2.51197299,  -0.90306273,
        -2.51197299,  -0.90306273,  -0.90306273,  -0.90306273,
        -0.90306273,  -6.29384529,  -0.90306273,  -0.90306273,
        -6.29384529,  -0.90306273,  -6.29384529,  -6.29384529,
        -6.29384529,  -6.29384529,  -6.29384529,  -6.29384529,
        -0.90306273,   4.8944664 ,  -6.29384529,  -0.90306273,
        -6.29384529,  -6.29384529,  -6.29384529,  -0.90306273,
        -0.90306273,  -0.90306273,  -0.90306273,   4.8944664 ,
        10.21597416,   4.8944664 ,  -0.90306273,  -0.90306273,
        -0.90306273,  -0.90306273,  -0.90306273,  -2.51197299,
        -2.51197299,  -2.51197299,  -0.90306273,  -0.90306273,
        -0.90306273,  -0.90306273,  -2.51197299,  10.21597416,
        -0.90306273,   4.8944664 ,  -0.90306273,  -0.90306273,
        -0.90306273,  -2.51197299,  -0.90306273,   4.89

In [83]:
# Add our error to our previous guess
# This is the end of 1 round of boosting
naive_guess += tree.predict(X)

In [85]:
gradient2 = y - naive_guess

In [86]:
# Gradient for round 2, should be a bit smaller than round 1
gradient2

0     -3.427273
1     -0.029744
2      1.951220
3      0.651220
4      3.451220
         ...   
501   -5.027273
502   -1.029744
503   -8.848780
504   -5.427273
505   -9.729744
Name: PRICE, Length: 506, dtype: float64

In [88]:
tree.fit(X,gradient2)

DecisionTreeRegressor(max_depth=4)

In [89]:
tree.predict(X)
# Numbers appear to be a bit smaller this time after round 2

array([-0.7878807 , -0.7878807 , -0.7878807 ,  1.44307887,  1.44307887,
        3.88472594, -0.7878807 ,  0.54781545, -1.93156374,  0.54781545,
        0.54781545, -0.7878807 ,  0.54781545, -0.7878807 , -0.7878807 ,
       -0.7878807 ,  3.844535  ,  0.54781545, -0.7878807 , -0.7878807 ,
        0.54781545, -0.7878807 ,  0.54781545,  0.54781545,  0.54781545,
        0.54781545,  0.54781545,  0.54781545, -0.7878807 , -0.7878807 ,
        0.54781545, -0.7878807 , -1.93156374,  0.54781545,  0.54781545,
       -0.7878807 , -0.7878807 , -0.7878807 ,  3.844535  ,  1.73639029,
        1.73639029,  1.73639029,  1.73639029,  1.73639029, -0.7878807 ,
       -0.7878807 , -0.7878807 ,  0.54781545, -1.93156374,  0.54781545,
       -0.7878807 , -0.7878807 ,  1.15555972,  1.15555972,  0.54781545,
        1.15555972, -0.7878807 , -0.7878807 , -0.98539764, -0.7878807 ,
       -0.7878807 , -0.7878807 , -0.7878807 , -0.7878807 ,  1.44307887,
        1.15555972,  1.15555972,  1.15555972, -0.7878807 ,  1.15

In [90]:
# Add our error to our previous guess
# This is the end of 2 round of boosting
naive_guess += tree.predict(X)


In [91]:
naive_guess # After round 2; values become more and more differentiated

array([26.63939202, 20.84186289, 31.96089979, 34.19185936, 34.19185936,
       25.51446953, 20.84186289, 20.56864878, 18.08926959, 20.56864878,
       20.56864878, 20.84186289, 20.56864878, 20.84186289, 20.84186289,
       20.84186289, 25.47427859, 16.78677649, 20.84186289, 20.84186289,
       16.78677649, 20.84186289, 16.78677649, 16.78677649, 16.78677649,
       16.78677649, 16.78677649, 16.78677649, 20.84186289, 26.63939202,
       16.78677649, 20.84186289, 14.3073973 , 16.78677649, 16.78677649,
       20.84186289, 20.84186289, 20.84186289, 25.47427859, 29.16366301,
       34.48517077, 29.16366301, 23.36613388, 23.36613388, 20.84186289,
       20.84186289, 20.84186289, 20.56864878, 18.08926959, 20.56864878,
       20.84186289, 20.84186289, 22.78530331, 22.78530331, 20.56864878,
       33.90434021, 20.84186289, 26.63939202, 20.64434595, 20.84186289,
       20.84186289, 19.23295263, 20.84186289, 26.63939202, 34.19185936,
       22.78530331, 22.78530331, 22.78530331, 20.84186289, 22.78

In [92]:
gradient3 = y - naive_guess

In [93]:
gradient3

0     -2.639392
1      0.758137
2      2.739100
3     -0.791859
4      2.008141
         ...   
501   -4.239392
502   -0.241863
503   -8.060900
504   -4.639392
505   -8.941863
Name: PRICE, Length: 506, dtype: float64

In [94]:

tree.fit(X,gradient3)

DecisionTreeRegressor(max_depth=4)

In [96]:
tree.predict(X)

array([-0.04749562,  1.31749276, -0.04749562, -0.04749562, -0.04749562,
        1.31749276, -1.68604327,  1.31749276, -1.68604327, -1.68604327,
        1.31749276, -1.68604327, -1.68604327, -1.56022518, -1.56022518,
       -1.56022518, -3.50683531, -1.56022518, -1.56022518, -1.56022518,
       -3.61531683, -1.56022518, -3.61531683, -1.56022518, -1.56022518,
       -1.56022518, -1.56022518, -1.56022518, -1.56022518, -3.61531683,
       -3.61531683, -3.61531683, -3.61531683, -3.61531683, -3.61531683,
        0.16784939,  0.16784939,  0.16784939,  0.16784939, -0.04749562,
       -0.04749562, -0.04749562,  1.31749276,  1.31749276,  1.31749276,
       -1.68604327, -1.68604327, -1.68604327, -1.68604327, -1.68604327,
       -1.68604327,  1.31749276,  1.31749276, -1.68604327, -1.56022518,
       -0.04749562,  1.31749276, -0.04749562,  1.31749276, -1.68604327,
       -1.68604327, -1.68604327,  1.31749276, -0.04749562, -0.04749562,
        1.31749276, -1.68604327, -1.68604327, -1.68604327, -1.68

In [97]:
# Add our error to our previous guess
# This is the end of 3 round of boosting
naive_guess += tree.predict(X)

In [99]:
naive_guess #Final predictions after 3 rounds.

array([26.59189641, 22.15935565, 31.91340417, 34.14436374, 34.14436374,
       26.83196229, 19.15581962, 21.88614154, 16.40322632, 18.88260551,
       21.88614154, 19.15581962, 18.88260551, 19.28163771, 19.28163771,
       19.28163771, 21.96744328, 15.22655131, 19.28163771, 19.28163771,
       13.17145966, 19.28163771, 13.17145966, 15.22655131, 15.22655131,
       15.22655131, 15.22655131, 15.22655131, 19.28163771, 23.0240752 ,
       13.17145966, 17.22654606, 10.69208047, 13.17145966, 13.17145966,
       21.00971228, 21.00971228, 21.00971228, 25.64212798, 29.1161674 ,
       34.43767516, 29.1161674 , 24.68362664, 24.68362664, 22.15935565,
       19.15581962, 19.15581962, 18.88260551, 16.40322632, 18.88260551,
       19.15581962, 22.15935565, 24.10279608, 21.09926005, 19.0084236 ,
       33.8568446 , 22.15935565, 26.59189641, 21.96183871, 19.15581962,
       19.15581962, 17.54690936, 22.15935565, 26.59189641, 34.14436374,
       24.10279608, 21.09926005, 21.09926005, 19.15581962, 21.09

In [101]:
naive_guess[:5]

array([26.59189641, 22.15935565, 31.91340417, 34.14436374, 34.14436374])

In [102]:
y[:5]

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
Name: PRICE, dtype: float64

In [107]:
gradient4 = y - naive_guess # From round 3 of predictions

In [108]:
tree.fit(X,gradient4)

DecisionTreeRegressor(max_depth=4)

In [110]:
learning_rate = .1

In [113]:
naive_guess += tree.predict(X) * learning_rate
# Typically add in this much smaller number

Importing gradient boosting!




In [114]:
from sklearn.ensemble import GradientBoostingRegressor

In [115]:
gbm = GradientBoostingRegressor()

In [117]:
gbm.get_params()
# These are the different knobs we can adjust

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'ls',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'presort': 'deprecated',
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

**learning_rate**: How much we scale errors predicted by to ensure updates are incremental

**max_depth**: How deep an individual tree will go

**n_estimators**: How many boosting rounds you will use

So defaults: Learning rate is .1 / trees 3 deep / 100 times

In [118]:
gbm.fit(X,y)

GradientBoostingRegressor()

In [121]:
gbm.score(X,y)
# Much better than an ordinary decision tree (R-Square of .88)


0.9761405838418584

In [122]:
# tells us how much each column contributed to the overall score
gbm.feature_importances_


array([0.02385397, 0.00045362, 0.0021715 , 0.00087275, 0.03667315,
       0.41118511, 0.00863911, 0.08448798, 0.00123333, 0.01152284,
       0.03455736, 0.01184085, 0.37250843])

In [123]:
X.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT'],
      dtype='object')